# Prepare Data
___

### Import Packages
___

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

### Read Data
___

In [2]:
demo_raw_df = pd.read_csv('../raw_data/demographic.csv')
hosp_raw_df = pd.read_csv('../raw_data/hospital.csv')
phys_raw_df = pd.read_csv('../raw_data/physiological.txt', sep='\t')
severity_raw_df = pd.read_json('../raw_data/severity.json')

We make a copy of each dataset to avoid making modifications to the original

In [3]:
demo_df = demo_raw_df.copy()
hosp_df = hosp_raw_df.copy()
phys_df = phys_raw_df.copy()
severity_df = severity_raw_df.copy()

### Demographic
___

#### Checking data
___

Before we start to modify and prepare the data, we will first check what type of data we are dealing with. 
We use .head to get an insight in what each dataset contains. 

In [4]:
demo_df.head()

,pasient_id,alder,kjønn,utdanning,inntekt,etnisitet
0,2,60.33899,female,12.0,$11-$25k,white
1,3,52.74698,female,12.0,under $11k,white
2,4,42.38498,female,11.0,under $11k,white
3,5,79.88495,female,NaN,NaN,white
4,6,93.01599,male,14.0,NaN,white


#### Checking types
___

We use .dtypes to check what type of value each column contains, we need to know the value type to know which operations we can use on the data. We cant take the mean of a string. 

In [5]:
demo_df.dtypes

pasient_id      int64
alder         float64
kjønn          object
utdanning     float64
inntekt        object
etnisitet      object
dtype: object

As we can see, both kjønn, inntekt and etnisitet have type object. These types need dummies to be used in the machine learning model. Salary could often be converted to int or float, however the way it is categorized in the data set, it makes more sense to give it dummies.

In [6]:
demo_df = pd.get_dummies(demo_df, columns=['inntekt', 'kjønn', 'etnisitet'], drop_first=True)
print(demo_df.head())

   pasient_id     alder  utdanning  inntekt_$25-$50k  inntekt_>$50k  \
0           2  60.33899       12.0             False          False   
1           3  52.74698       12.0             False          False   
2           4  42.38498       11.0             False          False   
3           5  79.88495        NaN             False          False   
4           6  93.01599       14.0             False          False   

   inntekt_under $11k  kjønn_male  etnisitet_black  etnisitet_hispanic  \
0               False       False            False               False   
1                True       False            False               False   
2                True       False            False               False   
3               False       False            False               False   
4               False        True            False               False   

   etnisitet_other  etnisitet_white  
0            False             True  
1            False             True  
2            F

#### Checking key stats
___

We will also use .describe to get key stats from the dataset. This is important to find flaws in data that we are given, and that needs to be cleaned. As an example we know that someone can not have an age of -1. When encountering invalid data there are multiple methods on how to handle them, the easiest way would be to remove the whole row. Another method is to switch the invalid value with NaN or null. 

When it comes to age it is also possible to switch the invalid data out with the mean or median, in cases where there are only a few instances of invalid data removing them could be fine, however if it is more common and a large pool of the dataset has invalid values imputing them would be preferred, if the latter is the case, i would switch it out with the mean. 

In [7]:
demo_raw_df.describe()

,pasient_id,alder,utdanning
count,7742.000000,7742.000000,6360.000000
mean,4539.500517,62.710904,11.752987
std,2637.118792,15.688113,3.458671
min,2.000000,-1.000000,0.000000
25%,2243.250000,52.915240,10.000000
50%,4538.500000,65.008000,12.000000
75%,6815.750000,74.131487,14.000000
max,9105.000000,101.847960,31.000000


As we can see the minimum age is -1, this is inavlid data and needs to be modified, if the datasets have few invalid instances these will be removed later, for now we will set the value to NaN to not corrupt furter analysis of statistical computations, like mean or median. 

In [8]:
demo_df.loc[demo_df['alder'] < 0, 'alder'] = np.nan

#### Checking missing data
___

We will be handling the missing data after the datasets are combined

In [9]:
demo_df.isnull().sum()

pasient_id               0
alder                    5
utdanning             1382
inntekt_$25-$50k         0
inntekt_>$50k            0
inntekt_under $11k       0
kjønn_male               0
etnisitet_black          0
etnisitet_hispanic       0
etnisitet_other          0
etnisitet_white          0
dtype: int64

#### Checking and removing duplicates
___

We also need to check for duplicate data, and remove it. 

In [10]:
duplicate_count_before = demo_df.duplicated().sum()
print(f"Duplicates before : {duplicate_count_before}")

demo_df = demo_df.drop_duplicates()

duplicate_count_after = demo_df.duplicated().sum()
print(f"Duplicates after : {duplicate_count_after}")

Duplicates before : 2
Duplicates after : 0


As we can see, we have removed two duplicates. 

We will now repeat the same process with the hospital data.

###  Hospital
___

#### Checking data
___

In [11]:
hosp_df.head()

,pasient_id,sykehusdød,oppholdslengde
0,2,1,4
1,3,0,17
2,4,0,3
3,5,0,-99
4,6,1,4


#### Checking types
___

In [12]:
hosp_df.dtypes

pasient_id        int64
sykehusdød        int64
oppholdslengde    int64
dtype: object

#### Checking key stats
___

In [13]:
hosp_df.describe()

,pasient_id,sykehusdød,oppholdslengde
count,7740.000000,7740.000000,7740.000000
mean,4540.665375,0.263178,17.728682
std,2636.463416,0.440387,21.995401
min,2.000000,0.000000,-99.000000
25%,2244.750000,0.000000,6.000000
50%,4539.500000,0.000000,11.000000
75%,6816.250000,1.000000,20.000000
max,9105.000000,1.000000,343.000000


As you can see, we have pasients with an oppholdslengde of less than zero, this is not possible, and since we are creating a macjine learning model that tries to predict oppholdslengde, we will remove these rows. 

In [14]:
hosp_df = hosp_df[hosp_df['oppholdslengde'] >= 0]
hosp_df.describe()

,pasient_id,sykehusdød,oppholdslengde
count,7734.000000,7734.000000,7734.000000
mean,4543.038014,0.263253,17.819240
std,2634.853898,0.440427,21.762188
min,2.000000,0.000000,3.000000
25%,2249.250000,0.000000,6.000000
50%,4541.500000,0.000000,11.000000
75%,6816.750000,1.000000,20.000000
max,9105.000000,1.000000,343.000000


Checking missing data

In [15]:
hosp_df.isnull().sum()

pasient_id        0
sykehusdød        0
oppholdslengde    0
dtype: int64

Checking and removing duplicates

In [16]:
duplicate_count_before = hosp_df.duplicated().sum()
print(f"Duplicates before : {duplicate_count_before}")

hosp_df = hosp_df.drop_duplicates()

duplicate_count_after = hosp_df.duplicated().sum()
print(f"Duplicates after : {duplicate_count_after}")

Duplicates before : 0
Duplicates after : 0


No duplicates

###  Physiological
___

#### Checking data
___

In [17]:
phys_df.head()

,pasient_id,blodtrykk,hvite_blodlegemer,hjertefrekvens,respirasjonsfrekvens,kroppstemperatur,lungefunksjon,serumalbumin,bilirubin,kreatinin,natrium,blod_ph,glukose,blodurea_nitrogen,urinmengde
0,2,43.0,17.097656,112.0,34.0,34.59375,98.00000,NaN,NaN,5.500000,132.0,7.250000,NaN,NaN,NaN
1,3,70.0,8.500000,88.0,28.0,37.39844,231.65625,NaN,2.199707,2.000000,134.0,7.459961,NaN,NaN,NaN
2,4,75.0,9.099609,88.0,32.0,35.00000,NaN,NaN,NaN,0.799927,139.0,NaN,NaN,NaN,NaN
3,5,59.0,13.500000,112.0,20.0,37.89844,173.31250,NaN,NaN,0.799927,143.0,7.509766,NaN,NaN,NaN
4,6,110.0,10.398438,101.0,44.0,38.39844,266.62500,NaN,NaN,0.699951,140.0,7.659180,NaN,NaN,NaN


#### Checking types
___

In [18]:
phys_df.dtypes

pasient_id                int64
blodtrykk               float64
hvite_blodlegemer       float64
hjertefrekvens          float64
respirasjonsfrekvens    float64
kroppstemperatur        float64
lungefunksjon           float64
serumalbumin            float64
bilirubin               float64
kreatinin               float64
natrium                 float64
blod_ph                 float64
glukose                 float64
blodurea_nitrogen       float64
urinmengde              float64
dtype: object

#### Checking key stats
___

In [19]:
phys_df.describe()

,pasient_id,blodtrykk,hvite_blodlegemer,hjertefrekvens,respirasjonsfrekvens,kroppstemperatur,lungefunksjon,serumalbumin,bilirubin,kreatinin,natrium,blod_ph,glukose,blodurea_nitrogen,urinmengde
count,7740.000000,7740.000000,7565.000000,7740.000000,7740.000000,7740.000000,5796.000000,4891.000000,5544.000000,7683.000000,7740.000000,5828.000000,3917.000000,4048.000000,3627.000000
mean,4540.665375,84.412468,12.302026,97.283940,23.348450,37.116579,239.608379,2.952622,2.565599,1.790278,137.574160,7.415439,160.031146,32.582040,2185.398372
std,2636.463416,27.713746,9.108076,31.499505,9.549561,1.257278,109.942808,0.889463,5.318276,1.711926,6.029887,0.080844,87.828819,26.826593,1458.900940
min,2.000000,0.000000,0.000000,0.000000,0.000000,31.699220,12.000000,0.399963,0.099991,0.099991,110.000000,6.829102,0.000000,1.000000,0.000000
25%,2244.750000,63.000000,6.899414,72.000000,18.000000,36.195310,155.093750,2.399902,0.500000,0.899902,134.000000,7.379883,103.000000,14.000000,1175.000000
50%,4539.500000,77.000000,10.599609,100.000000,24.000000,36.695310,223.984375,2.899902,0.899902,1.199951,137.000000,7.419922,135.000000,23.000000,1963.000000
75%,6816.250000,107.000000,15.298828,120.000000,28.000000,38.195310,304.750000,3.599609,1.899902,1.899902,141.000000,7.469727,190.000000,43.000000,2980.000000
max,9105.000000,195.000000,128.781250,232.000000,90.000000,41.695310,869.375000,29.000000,63.000000,21.500000,181.000000,7.769531,1092.000000,192.000000,9000.000000


#### Checking missing data
___

In [20]:
phys_df.isnull().sum()

pasient_id                 0
blodtrykk                  0
hvite_blodlegemer        175
hjertefrekvens             0
respirasjonsfrekvens       0
kroppstemperatur           0
lungefunksjon           1944
serumalbumin            2849
bilirubin               2196
kreatinin                 57
natrium                    0
blod_ph                 1912
glukose                 3823
blodurea_nitrogen       3692
urinmengde              4113
dtype: int64

#### Checking and removing duplicates
___

In [21]:
duplicate_count_before = phys_df.duplicated().sum()
print(f"Duplicates before : {duplicate_count_before}")

phys_df = phys_df.drop_duplicates()

duplicate_count_after = phys_df.duplicated().sum()
print(f"Duplicates after : {duplicate_count_after}")

Duplicates before : 0
Duplicates after : 0


No duplicates

###  Severity
___

#### Checking data
___

In [22]:
severity_df.head()

,sykdomskategori_id,sykdomskategori,pasient_id,dødsfall,sykdom_underkategori,antall_komorbiditeter,koma_score,adl_pasient,adl_stedfortreder,fysiologisk_score,apache_fysiologisk_score,overlevelsesestimat_2mnd,overlevelsesestimat_6mnd,diabetes,demens,kreft,lege_overlevelsesestimat_2mnd,lege_overlevelsesestimat_6mnd,dnr_status,dnr_dag
0,A1s,ARF/MOSF,"[5, 15, 18, 23, 28, 34, 39, 43, 46, 47, 48, 58...","[0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, ...","[ARF/MOSF w/Sepsis, ARF/MOSF w/Sepsis, ARF/MOS...","[1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 2, 0, 3, 1, 3, ...","[26.0, 26.0, 26.0, 0.0, 26.0, 37.0, 0.0, 0.0, ...","[None, None, None, None, None, None, None, Non...","[2.0, None, 0.0, 5.0, 2.0, None, 0.0, None, No...","[23.5, 30.5, 40.296875, 31.6992188, 46.796875,...","[30.0, 39.0, 58.0, 42.0, 85.0, 49.0, 5.0, 76.0...","[0.6348876950000001, 0.590942383, 0.2129821780...","[0.5329589840000001, 0.481994629, 0.1169891360...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[no, no, no, no, yes, no, no, no, no, no, yes,...","[0.899999619, 0.899999619, 0.09999996400000001...","[0.9, 0.9, 0.001, 0.5, 0.000125, 0.60000000000...","[None, None, None, None, None, None, None, Non...","[None, None, None, None, None, None, None, Non..."
1,BrY,COPD/CHF/Cirrhosis,"[2, 3, 7, 8, 11, 13, 14, 19, 20, 30, 31, 32, 3...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, ...","[Cirrhosis, Cirrhosis, CHF, CHF, CHF, Cirrhosi...","[2, 2, 1, 3, 1, 1, 0, 2, 1, 2, 2, 2, 2, 1, 1, ...","[44.0, 0.0, 0.0, 26.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[None, 1.0, 0.0, None, 2.0, 0.0, 0.0, 7.0, 3.0...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 7.0, None,...","[52.6953125, 20.5, 17.296875, 21.5976562, 14.5...","[74.0, 45.0, 46.0, 53.0, 14.0, 30.0, 34.0, 42....","[0.000999928, 0.790893555, 0.892944336, 0.6708...","[0.0, 0.6649169920000001, 0.820922852, 0.49896...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[no, no, no, no, no, no, no, no, no, no, no, n...","[0.0, 0.75, None, 0.799999714, 0.699999809, No...","[0.0, 0.5, 0.7000000000000001, 0.4, 0.5, None,...","[None, None, None, None, None, None, None, Non...","[None, None, None, None, None, None, None, Non..."
2,ChE,Cancer,"[4, 9, 10, 12, 16, 17, 21, 24, 27, 41, 42, 54,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, ...","[Lung Cancer, Lung Cancer, Colon Cancer, Lung ...","[2, 2, 0, 0, 1, 2, 0, 0, 0, 1, 1, 1, 2, 0, 1, ...","[0.0, 26.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 44.0...","[0.0, None, 0.0, 1.0, 2.0, None, 0.0, None, No...","[0.0, 7.0, None, 1.0, 0.0, None, 0.0, None, No...","[20.0976562, 15.8984375, 2.2998047, 16.3984375...","[19.0, 17.0, 9.0, 17.0, 11.0, 4.0, 16.0, 11.0,...","[0.6989746090000001, 0.570922852, 0.9528808590...","[0.411987305, 0.24899292, 0.8879394530000001, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[metastatic, metastatic, metastatic, metastati...","[0.899999619, 0.049999982000000005, None, None...","[0.5, 0.000125, None, None, 0.7000000000000001...","[None, dnr ved innleggelse, None, None, None, ...","[None, 0.0, None, None, None, None, None, None..."
3,DWw,Coma,"[6, 162, 188, 250, 252, 262, 275, 309, 323, 35...","[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, ...","[Coma, Coma, Coma, Coma, Coma, Coma, Coma, Com...","[1, 0, 0, 2, 1, 2, 1, 0, 0, 1, 1, 0, 2, 2, 0, ...","[55.0, 61.0, 94.0, 55.0, 94.0, 100.0, 100.0, 5...","[None, None, None, None, None, None, None, Non...","[1.0, None, None, 1.0, None, None, None, None,...","[19.3984375, 30.3984375, 20.296875, 30.8984375...","[27.0, 36.0, 22.0, 53.0, 40.0, 25.0, 58.0, 16....","[0.28497314500000004, 0.438964844, 0.280944824...","[0.214996338, 0.365966797, 0.211975098, 0.2729...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[no, no, no, no, yes, no, no, no, no, no, no, ...","[0.0, None, None, None, None, 0.09999996400000...","[0.0, None, N

#### Checking types and modifying
___

In [23]:
severity_df.dtypes

sykdomskategori_id               object
sykdomskategori                  object
pasient_id                       object
dødsfall                         object
sykdom_underkategori             object
antall_komorbiditeter            object
koma_score                       object
adl_pasient                      object
adl_stedfortreder                object
fysiologisk_score                object
apache_fysiologisk_score         object
overlevelsesestimat_2mnd         object
overlevelsesestimat_6mnd         object
diabetes                         object
demens                           object
kreft                            object
lege_overlevelsesestimat_2mnd    object
lege_overlevelsesestimat_6mnd    object
dnr_status                       object
dnr_dag                          object
dtype: object

We need to modify the objects in severity that contains lists to not contain lists, we can use .explode. When we used heads we could see that each column expect the first 2 needs to be used explode on. 

In [24]:
columns_need_explode = severity_df.columns[2:].tolist()
severity_df = severity_df.explode(columns_need_explode)
severity_df.head()

,sykdomskategori_id,sykdomskategori,pasient_id,dødsfall,sykdom_underkategori,antall_komorbiditeter,koma_score,adl_pasient,adl_stedfortreder,fysiologisk_score,apache_fysiologisk_score,overlevelsesestimat_2mnd,overlevelsesestimat_6mnd,diabetes,demens,kreft,lege_overlevelsesestimat_2mnd,lege_overlevelsesestimat_6mnd,dnr_status,dnr_dag
0,A1s,ARF/MOSF,5,0,ARF/MOSF w/Sepsis,1,26.0,None,2.0,23.5,30.0,0.634888,0.532959,0,0,no,0.9,0.9,None,None
0,A1s,ARF/MOSF,15,0,ARF/MOSF w/Sepsis,1,26.0,None,None,30.5,39.0,0.590942,0.481995,0,0,no,0.9,0.9,None,None
0,A1s,ARF/MOSF,18,1,ARF/MOSF w/Sepsis,0,26.0,None,0.0,40.296875,58.0,0.212982,0.116989,0,0,no,0.1,0.001,None,None
0,A1s,ARF/MOSF,23,0,ARF/MOSF w/Sepsis,1,0.0,None,5.0,31.699219,42.0,0.650879,0.55188,0,0,no,0.6,0.5,None,None
0,A1s,ARF/MOSF,28,1,MOSF w/Malig,1,26.0,None,2.0,46.796875,85.0,0.021,0.004999,0,0,yes,0.05,0.000125,None,None


All columns has type object, this needs to be modified before we can use them in the machine learning model. 
Following the list, describing type of variables, pasient_id, dødsfall, antall_komorbiditeter, diabets and demens should be int.

In [25]:
severity_df['pasient_id'] = pd.to_numeric(severity_df['pasient_id'], errors='coerce').astype('Int64')
severity_df['dødsfall'] = pd.to_numeric(severity_df['dødsfall'], errors='coerce').astype('Int64')
severity_df['antall_komorbiditeter'] = pd.to_numeric(severity_df['antall_komorbiditeter'], errors='coerce').astype('Int64')
severity_df['diabetes'] = pd.to_numeric(severity_df['diabetes'], errors='coerce').astype('Int64')
severity_df['demens'] = pd.to_numeric(severity_df['demens'], errors='coerce').astype('Int64')

Categorical variables like sykdomskategori, sykdom_underkategori, kreft, and dnr_status will be given dummy variables.

In [26]:
severity_df = pd.get_dummies(severity_df, columns=['sykdomskategori_id', 'sykdomskategori', 'sykdom_underkategori', 
                                                   'kreft', 'dnr_status'])
severity_df.head()

,pasient_id,dødsfall,antall_komorbiditeter,koma_score,adl_pasient,adl_stedfortreder,fysiologisk_score,apache_fysiologisk_score,overlevelsesestimat_2mnd,overlevelsesestimat_6mnd,...,sykdom_underkategori_Cirrhosis,sykdom_underkategori_Colon Cancer,sykdom_underkategori_Coma,sykdom_underkategori_Lung Cancer,sykdom_underkategori_MOSF w/Malig,kreft_metastatic,kreft_no,kreft_yes,dnr_status_dnr før innleggelse,dnr_status_dnr ved innleggelse
0,5,0,1,26.0,None,2.0,23.5,30.0,0.634888,0.532959,...,False,False,False,False,False,False,True,False,False,False
0,15,0,1,26.0,None,None,30.5,39.0,0.590942,0.481995,...,False,False,False,False,False,False,True,False,False,False
0,18,1,0,26.0,None,0.0,40.296875,58.0,0.212982,0.116989,...,False,False,False,False,False,False,True,False,False,False
0,23,0,1,0.0,None,5.0,31.699219,42.0,0.650879,0.55188,...,False,False,False,False,False,False,True,False,False,False
0,28,1,1,26.0,None,2.0,46.796875,85.0,0.021,0.004999,...,False,False,False,False,True,False,False,True,False,False


Continuous variables like koma_score, adl_pasient, adl_stedfortreder,fysiologisk_score, apache_fysiologisk_score, overlevelsesestimat_2mnd,
overlevelsesestimat_6mnd, lege_overlevelsesestimat_2mnd, lege_overlevelsesestimat_6mnd and dnr_dag should be float

In [27]:
severity_df['koma_score'] = pd.to_numeric(severity_df['koma_score'], errors='coerce')
severity_df['adl_pasient'] = pd.to_numeric(severity_df['adl_pasient'], errors='coerce')
severity_df['adl_stedfortreder'] = pd.to_numeric(severity_df['adl_stedfortreder'], errors='coerce')
severity_df['fysiologisk_score'] = pd.to_numeric(severity_df['fysiologisk_score'], errors='coerce')
severity_df['apache_fysiologisk_score'] = pd.to_numeric(severity_df['apache_fysiologisk_score'], errors='coerce')
severity_df['overlevelsesestimat_2mnd'] = pd.to_numeric(severity_df['overlevelsesestimat_2mnd'], errors='coerce')
severity_df['overlevelsesestimat_6mnd'] = pd.to_numeric(severity_df['overlevelsesestimat_6mnd'], errors='coerce')
severity_df['lege_overlevelsesestimat_2mnd'] = pd.to_numeric(severity_df['lege_overlevelsesestimat_2mnd'], errors='coerce')
severity_df['lege_overlevelsesestimat_6mnd'] = pd.to_numeric(severity_df['lege_overlevelsesestimat_6mnd'], errors='coerce')
severity_df['dnr_dag'] = pd.to_numeric(severity_df['dnr_dag'], errors='coerce')

#### Checking missing data
___

In [28]:
phys_df.isnull().sum()

pasient_id                 0
blodtrykk                  0
hvite_blodlegemer        175
hjertefrekvens             0
respirasjonsfrekvens       0
kroppstemperatur           0
lungefunksjon           1944
serumalbumin            2849
bilirubin               2196
kreatinin                 57
natrium                    0
blod_ph                 1912
glukose                 3823
blodurea_nitrogen       3692
urinmengde              4113
dtype: int64

#### Checking and removing duplicates
___

In [29]:
duplicate_count_before = phys_df.duplicated().sum()
print(f"Duplicates before : {duplicate_count_before}")

phys_df = phys_df.drop_duplicates()

duplicate_count_after = phys_df.duplicated().sum()
print(f"Duplicates after : {duplicate_count_after}")

Duplicates before : 0
Duplicates after : 0


No duplicates

## Making One Data Set
___

In [30]:
demo_hosp_df = pd.merge(demo_df, hosp_df, on='pasient_id')
demo_hosp_phys_df = pd.merge(demo_hosp_df, phys_df, on='pasient_id')
pasient_id_df = pd.merge(demo_hosp_phys_df, severity_df, on='pasient_id')

In [31]:
pasient_id_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7734 entries, 0 to 7733
Data columns (total 62 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   pasient_id                              7734 non-null   int64  
 1   alder                                   7729 non-null   float64
 2   utdanning                               6354 non-null   float64
 3   inntekt_$25-$50k                        7734 non-null   bool   
 4   inntekt_>$50k                           7734 non-null   bool   
 5   inntekt_under $11k                      7734 non-null   bool   
 6   kjønn_male                              7734 non-null   bool   
 7   etnisitet_black                         7734 non-null   bool   
 8   etnisitet_hispanic                      7734 non-null   bool   
 9   etnisitet_other                         7734 non-null   bool   
 10  etnisitet_white                         7734 non-null   bool

## Handling NaNs
___

### Colums with NaNs
___

To find columns with NaNs we use .isnull, we also want to find count of NaNs in column and the percentage of NaNs for each column.

In [32]:
nan_counts = pasient_id_df.isnull().sum()
nan_percentage = (nan_counts / len(pasient_id_df)) * 100
nan_summary = pd.DataFrame({
    'NaN Count': nan_counts,
    'Percentage': nan_percentage
})

nan_summary = nan_summary[nan_summary['NaN Count'] > 0]

print(nan_summary)

                               NaN Count  Percentage
alder                                  5    0.064650
utdanning                           1380   17.843289
hvite_blodlegemer                    175    2.262736
lungefunksjon                       1943   25.122834
serumalbumin                        2847   36.811482
bilirubin                           2194   28.368244
kreatinin                             57    0.737005
blod_ph                             1911   24.709077
glukose                             3817   49.353504
blodurea_nitrogen                   3686   47.659685
urinmengde                          4107   53.103181
adl_pasient                         4792   61.960176
adl_stedfortreder                   2440   31.549004
lege_overlevelsesestimat_2mnd       1419   18.347556
lege_overlevelsesestimat_6mnd       1405   18.166537
dnr_dag                             6664   86.164986


In handling the missing data, we will address some columns immediately where the fixes are straightforward. For instance, columns with minimal missing values can have rows removed, or straightforward imputations can be applied. However, other columns with a significant amount of missing data will require further consideration. These will be handled with imputation techniques or other strategies, which will be implemented in the modeling section to ensure the integrity of the analysis.

### Alder
___

Alder only has 5 NaN values, since the count is so low, we will just remove these iDs.

In [33]:
nan_count_before = pasient_id_df['alder'].isna().sum()
print(f"NaN count for 'alder' before dropping: {nan_count_before}")

pasient_id_df = pasient_id_df.dropna(subset=['alder'])

nan_count_after = pasient_id_df['alder'].isna().sum()
print(f"NaN count for 'alder' after dropping: {nan_count_after}")

NaN count for 'alder' before dropping: 5
NaN count for 'alder' after dropping: 0


### Utdanning
___

Signifcant amount of missing data, will be handled in modeling.

### Physiological factors
___

All together the physiological colums have signifcant amount of missing data, and will be handled in modeling. 

### Adl_pasient and Adl_stedfortreder
___

Significant amount of missing data, will be handled in modeling.

### Lege_overlevelsesestimat_2mnd and ege_overlevelsesestimat_6mnd
___

Significant amount of missing data, will be handled in modeling.

### Dnr_dag
___

Significant amount of missing data, will be handled in modeling.

## Splitting in to train-, test- and validationdata
___

We use train_test_split to split pasient_id_df into train and test datasets, and then further split the train dataset into train and val datasets.
train_df: 60%,
val_df: 20%,
test_df: 20%. We then store them in the folder cleaned_data

In [34]:
X = pasient_id_df.drop(['oppholdslengde', 'pasient_id'], axis=1)
y = pasient_id_df['oppholdslengde']

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=1)

X_train.to_csv('../cleaned_data/X_train.csv', index=False)
y_train.to_csv('../cleaned_data/y_train.csv', index=False)

X_val.to_csv('../cleaned_data/X_val.csv', index=False)
y_val.to_csv('../cleaned_data/y_val.csv', index=False)

X_test.to_csv('../cleaned_data/X_test.csv', index=False)
y_test.to_csv('../cleaned_data/y_test.csv', index=False)